### Múltiple cárga de datos

In [1]:
# tools
from utils import *

In [2]:
# Multiple datasets
mex_industry = pd.read_csv('emp_mex_industry.csv')
data_denue = pd.read_csv('sector_empresa.csv')
padron_sat = pd.read_csv('pad_exp_sec.csv')

In [3]:
# Colum names for each dataset
print(f"Padron del SAT{padron_sat.columns}")
print(f"Empresas del DENUE{data_denue.columns}")
print(f"Empresas de Mex Industry{mex_industry.columns}")

Padron del SATIndex(['ID', 'RFC', 'NOMBRE', 'FECHA', 'SECTOR', 'DESCRIPCIÓN'], dtype='object')
Empresas del DENUEIndex(['clase_act_id', 'empresa', 'clase_act', 'estrato', 'calle',
       'numero_ext', 'colonia', 'cp', 'ubicacion', 'longitud', 'latitud',
       'fecha_alta', 'telefono', 'correo'],
      dtype='object')
Empresas de Mex IndustryIndex(['Razon_social', 'Direccion'], dtype='object')


In [7]:
import string

# Función de limpieza de caracteres especiales
# data cleaning function
puntuacion = string.punctuation

def limpiar_titulos(columna):
    snpunct = str.maketrans('', '', string.punctuation)
    titulo_limp = columna.translate(snpunct)
    return titulo_limp

In [12]:
# Extraer datos según la ubicación
# Data extraction according to location
denue_nl_coah = data_denue[data_denue['ubicacion'].str.contains('COAHUILA DE ZARAGOZA|NUEVO LEÓN', case=False, regex=True)]
denue_nl_coah['empresa'] = denue_nl_coah['empresa'].apply(limpiar_titulos)
denue_nl_coah.shape

C:\Users\Adrián Vázquez\AppData\Local\Temp\ipykernel_20868\1502413525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  denue_nl_coah['empresa'] = denue_nl_coah['empresa'].apply(limpiar_titulos)


(1614, 14)

In [13]:
# Texto en minusculas
# Lowercase text
mex_industry['Razon_social'] = mex_industry['Razon_social'].str.lower()
data_denue['empresa'] = data_denue['empresa'].str.lower()
padron_sat['NOMBRE'] = padron_sat['NOMBRE'].str.lower()
# Limpieza de carácteres especiales
# Special characters cleaning
mex_industry['Razon_social'] = mex_industry['Razon_social'].apply(limpiar_titulos)
data_denue['empresa'] = data_denue['empresa'].apply(limpiar_titulos)
padron_sat['NOMBRE'] = padron_sat['NOMBRE'].apply(limpiar_titulos)

In [11]:
val1 = mex_industry['Razon_social'].unique()
val2 = denue_nl_coah['empresa'].unique()
val3 = padron_sat['NOMBRE'].unique()
# Eliminar coincidencias encontradas en los data sets
# Removing match rows
coincidentes0 = set(val2) & set(val1)
coincidentes1 = set(val2) & set(val3)

coincidentes = list(coincidentes0) + list(coincidentes1)
len(coincidentes)

22

In [16]:
# Filtrado de datos: filas coincidentes y separación de datos de la columna "ubicacion_completa"
filt_denue = denue_nl_coah[~denue_nl_coah['empresa'].isin(coincidentes)]
filt_denue["ubicacion_completa"] = filt_denue["calle"]+', '+filt_denue["numero_ext"].astype(str)+ ', '+filt_denue["colonia"]+ ', '+filt_denue["cp"].astype(str)+ ', '+filt_denue["ubicacion"]
filt_denue['empresa'] = filt_denue['empresa'].str.lower()
filt_denue.to_csv('fil_nl_coah_denue.csv', index=False)

C:\Users\Adrián Vázquez\AppData\Local\Temp\ipykernel_20868\64842652.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_denue["ubicacion_completa"] = filt_denue["calle"]+', '+filt_denue["numero_ext"].astype(str)+ ', '+filt_denue["colonia"]+ ', '+filt_denue["cp"].astype(str)+ ', '+filt_denue["ubicacion"]
C:\Users\Adrián Vázquez\AppData\Local\Temp\ipykernel_20868\64842652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_denue['empresa'] = filt_denue['empresa'].str.lower()


In [94]:
# Para extraer el RFC de la tabla "padron_sat" solo seleccionamos aquellas filas que se encuentran dentro de los nombres coincidentes en el DENUE
rfc_sat = padron_sat[padron_sat['NOMBRE'].isin(coincidentes)]
rfc_sat = rfc_sat.iloc[:,1:3]

In [56]:
# filtrado de los datos del DENUE con los coincidentes encontrados en las dos tablas
base_f = data_denue[data_denue['empresa'].isin(coincidentes)]
# Union de columnas para crear la dirección completa
base_f["ubicacion_completa"] = base_f["calle"]+', '+base_f["numero_ext"].astype(str)+ ', '+base_f["colonia"]+ ', '+base_f["cp"].astype(str)+ ', '+base_f["ubicacion"]
# Selección de las columnas necesarias para crear la base final
base_fin = base_f[["clase_act_id","empresa","clase_act","estrato","ubicacion_completa","fecha_alta","telefono","correo"]]
base_fin

In [58]:
# Guardado de la tabla final
base_fin.to_excel('base_fin.xlsx', index=False)

In [59]:
# Cargamos ubaerbase para seleccionar solo aquellos nombres de empresas que no se encuentren en esta base
uberbase = pd.read_csv('uberbase.csv')
uberbase

,Unnamed: 0,Razon_social,Nombre_comercial,Sector,Telefono,Correo,Web,Ubicacion,Domicilio
0,5,FOOD SERVICE DE MEXICO SA DE CV,NaN,Industria alimentaria,871757 5202,NaN,NaN,Coahuila,Carretera\nMonterrey-Saltillo Km63 N°5230 Col....
1,6,INTERCARNES COAHUILA,NaN,Industria alimentaria,871 720 0022,obuentello@bafar.com.mx,NaN,Coahuila,Av Pdte Carranza N° 3190 Col. Cuarto de Cobián...
2,7,LECHE BELL SA DE CV,NaN,Industria alimentaria,871 229 0800,contacto@lechebell.com,NaN,Coahuila,Domingo Valdez Llano N° 200 Ciudad Industrial ...
3,8,ALKOSA ALIMENTOS KONGELADOS SOL SA DE CV,NaN,Industria alimentaria,867 308 3070,NaN,NaN,Coahuila,"México, Los Pinos N°int 57 Ext. 1101, Allende..."
4,9,"AYVI, S. A. DE C. V.",NaN,Industria alimentaria,815030-2400,ventas@ayvi.com.mx,NaN,Coahuila,NaN
...,...,...,...,...,...,...,...,...,...
239,244,"KAR FRI ALIMENTOS, S. A. DE C. V.",NaN,Industria alimentaria,81 83302856,hgabriel@karfri.com,NaN,Monterrey,Gonzalitos
240,245,PRODUCTOS DESHIDRATADOS PAGA S.A DE C.V,NaN,Industria alimentaria,8115053551 o 8110 93 73 24,contabilidad@musiquerascom.mx,NaN,Monterrey,Arturo Carranza
241,246,ALIMENTOS NUTRI FACIL SA CV,NaN,Fabricación de maquinaria y equipo,81 8122-0600,datospersonales@nutrifacil.com.mx,NaN,Monterrey,Privada Martín de Zavala
242,247,JP METALMECANICA APLICADA,NaN,"Fabricación de productos textiles, excepto pre...",(81) 1090-8083 (81) 1090-9438,amorales@jpmetalmecanica.com.mx,NaN,Monterrey,Puerto La Paz


In [60]:
uberbase['Razon_social'] = uberbase['Razon_social'].str.lower()
uberbase['Razon_social'] = uberbase['Razon_social'].apply(limpiar_titulos)
val1 = base_fin['empresa'].unique()
val2 = uberbase['Razon_social'].unique()
coincidente = set(val2) & set(val1)
print(f" En la tabla se encuentran {len(coincidente)} coincidencias: {coincidente}")

7

In [97]:
filtro_est = base_f[base_f['ubicacion'].str.contains('COAHUILA DE ZARAGOZA|NUEVO LEÓN', case=False, regex=True)]

In [101]:
rfc_sat.rename(columns={'NOMBRE':'empresa'}, inplace=True)

empresas_filtradas_rfc = filtro_est.merge(rfc_sat, on='empresa', how='left')

In [103]:
valor1 = empresas_filtradas_rfc['empresa'].unique()
valor2 = uberbase['Razon_social'].unique()
coin = set(valor1) & set(valor2)
print(coin)

{'brembo mexico sa de cv', 'conmet de mexico sa de cv', 'contitech fluid mexicana sa de cv', 'consorcio industrial mexicano de autopartes sa de cv', 'accuride del norte sa de cv', 'cemm mex sa de cv', 'ensambles derramadero s de rl de cv'}


In [110]:
empresas_unicas_filtradas = empresas_filtradas_rfc.drop_duplicates(subset=['RFC'], keep = 'first')
empresas_unicas_filtradas.to_excel('empresas_filtradas_rfc.xlsx', index=False)
empresas_unicas_filtradas

,clase_act_id,empresa,clase_act,estrato,calle,numero_ext,colonia,cp,ubicacion,longitud,latitud,fecha_alta,telefono,correo,RFC
0,434221,aceros levinson sa de cv,Comercio al por mayor de materiales metálicos ...,51 a 100 personas,ADOLFO RUIZ CORTINES,1824.0,GARZA NIETO,64420.0,"MONTERREY, Monterrey, NUEVO LEÓN",-100.330476,25.704370,2010-07,NaN,ACEROSLEV@INTERCABLE.NET.MX,ALE870612DS8
1,333910,aim servicios administrativos s de rl de cv,Fabricación de bombas y sistemas de bombeo,51 a 100 personas,NINGUNO,402.0,LAS AMERICAS,66640.0,"CIUDAD APODACA, Apodaca, NUEVO LEÓN",-100.186425,25.745355,2010-07,NaN,PTIJERINA@AIIG.COM,ASA051215465
7,466112,aire acondicionado proyectos sa de cv,Comercio al por menor de electrodomésticos men...,51 a 100 personas,PASEO DE LOS LEONES,341.0,MITRAS CENTRO,64460.0,"MONTERREY, Monterrey, NUEVO LEÓN",-100.341812,25.696264,2010-07,NaN,INFO@ACPROYECTOS.COM,AAP010601S21
8,325610,alen del norte sa de cv,"Fabricación de jabones, limpiadores y dentífricos",51 a 100 personas,MONTERREY - COLOMBIA,0.0,LA COLONIA LA VIDRIERA,65070.0,"LA COLONIA (VIDRIERA), Lampazos de Naranjo, NU...",-100.514669,27.000303,2016-10,NaN,NaN,ANX940223JQ0
12,467111,apm sujeciones y construcciones sa de cv,Comercio al por menor en ferreterías y tlapale...,51 a 100 personas,LOS GIRASOLES,104.0,LOS GIRASOLES,66050.0,"CIUDAD GENERAL ESCOBEDO, General Escobedo, NUE...",-100.325611,25.787433,2010-07,NaN,ibarboza@cscontadores.com,NaN
14,334410,acutran de mexico sa de cv,Fabricación de componentes electrónicos,101 a 250 personas,CHIHUAHUA,0.0,ACUÑA,26248.0,"CIUDAD ACUÑA, Acuña, COAHUILA DE ZARAGOZA",-100.985279,29.338941,2010-07,NaN,BQUIROZ@SNCMFG.COM,AME9202183Q7
18,335920,adirondack wire cable s de rl de cv,Fabricación de cables de conducción eléctrica,101 a 250 personas,ADOLFO RUÍZ CORTINES,1855.0,ESTRELLA,64400.0,"MONTERREY, Monterrey, NUEVO LEÓN",-100.330543,25.704671,2014-12,NaN,ORTEGA_OSCAR@WBLAKE.COM,AW&0511299S9
25,336390,accuride del norte sa de cv,Fabricación de otras partes para vehículos aut...,251 y más personas,LAS AMERICAS,301.0,LAS AMERICAS,65550.0,"LAS AMERICAS [PARQUE INDUSTRIAL], Ciénega de F...",-100.228454,25.946441,2010-07,NaN,NaN,ANO070531QR7
27,336320,aees manufacturera s de rl de cv,Fabricación de equipo eléctrico y electrónico ...,251 y más personas,PRESA LA AMISTAD,0.0,LA AMISTAD,26220.0,"CIUDAD ACUÑA, Acuña, COAHUILA DE ZARAGOZA",-100.960973,29.333200,2014-12,NaN,NaN,AFM010102MQ5
31,325220,akra polyester sa de cv,Fabricación de fibras químicas,251 y más personas,ROBLE,1.0,PEDRO LOZANO,64400.0,"MONTERREY, Monterrey, NUEVO LEÓN",-100.334825,25.706894,2010-07,NaN,DANIELA.VARGAS@ALLIAX.COM,APO5912176H6
